In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import itertools 
import multiprocessing as mp
import numpy as np
import sys

sys.path.append('../')
from baseModels.SVM.model import SVM

In [2]:
feature_dir = '../featEngg/pssmMethods/data/featvec/'
feature_files = [f.name for f in os.scandir(feature_dir) if f.name.endswith('.csv.gz')]

In [3]:
label_file = '../data/LabelFiles/EnzymeLabelsMultiClass.csv'

In [4]:
def model_evaluate(filename,random_seed):
    df1 = pd.read_csv(feature_dir+filename,header=None)
    df2 = pd.read_csv(label_file,header=None)
    df = df1.merge(df2,on=0)
    enz_names = df[0].values
    X = df.iloc[:,1:-1].values
    y = df.iloc[:,-1].values
    if X.shape[1]<55:
        n_comp = int(0.75*X.shape[1])
    else:
        n_comp=55
    X_train, X_test, y_train, y_test,enz_train,enz_test = train_test_split(X, y,enz_names, test_size=0.25, random_state=random_seed)
    svm = SVM(X_train,X_test,y_train,y_test,verbose=False,optimize=False, pca_comp=n_comp,kern='rbf',regC=30)
    return svm.acc_train,svm.acc_test
 

In [5]:
mp.cpu_count()

24

In [6]:
pool = mp.Pool(mp.cpu_count())

In [7]:
iterable = list(itertools.product(feature_files,range(1000)))

In [8]:
%%time
model_accs = list(pool.starmap(model_evaluate,iterable))

CPU times: user 324 ms, sys: 99 ms, total: 423 ms
Wall time: 4min 28s


In [9]:
mydict = dict()

for m,acc in zip(iterable,model_accs):
    model_name=m[0].replace('.csv.gz','')
    seed = m[1]
    if model_name not in mydict:
        mydict[model_name] = {seed:acc}
    else:
        mydict[model_name][seed] = acc

In [10]:
def get_model_stats(mname):
    train_accs = [v[0] for k,v in mydict[mname].items()]
    test_accs = [v[1] for k,v in mydict[mname].items()]
    return {'train_min':min(train_accs),'train_mean':np.mean(train_accs),
            'train_max':max(train_accs),'train_std':np.std(train_accs),
            'test_min':min(test_accs),'test_mean':np.mean(test_accs),
            'test_max':max(test_accs),'test_std':np.std(test_accs)}

In [11]:
get_model_stats('rpssm')

{'train_min': 0.7674418604651163,
 'train_mean': 0.8419302325581396,
 'train_max': 0.9186046511627907,
 'train_std': 0.024064891091259488,
 'test_min': 0.3793103448275862,
 'test_mean': 0.6075862068965517,
 'test_max': 0.8620689655172413,
 'test_std': 0.08242461334250296}

In [12]:
results_dict= {mname:get_model_stats(mname) for mname in mydict.keys()}
    

In [13]:
results_df = pd.DataFrame(results_dict).T

In [14]:
results_df.to_csv('../data/SimResults/pssmResults.csv',index=True)

In [15]:
results_df.sort_values('train_mean')

,train_min,train_mean,train_max,train_std,test_min,test_mean,test_max,test_std
edp,0.651163,0.724140,0.802326,0.027130,0.379310,0.606690,0.862069,0.078762
tpc,0.558140,0.737233,1.000000,0.154779,0.310345,0.595586,0.827586,0.084971
aatp,0.732558,0.835547,0.965116,0.054382,0.413793,0.637138,0.862069,0.082250
rpssm,0.767442,0.841930,0.918605,0.024065,0.379310,0.607586,0.862069,0.082425
smoothed_pssm,0.779070,0.848965,0.918605,0.021613,0.275862,0.533931,0.793103,0.075357
aac_pssm,0.790698,0.856523,0.918605,0.022215,0.310345,0.642517,0.931034,0.083433
medp,0.802326,0.869233,0.930233,0.021121,0.379310,0.594862,0.862069,0.079280
eedp,0.802326,0.871337,0.930233,0.020907,0.379310,0.594690,0.862069,0.079186
ab_pssm,0.825581,0.893581,0.965116,0.024059,0.344828,0.590103,0.827586,0.078529
pssm_cc,0.837209,0.899814,0.965116,0.022144,0.379310,0.634552,0.896552,0.081677
